In [ ]:
!pip install -q transformers

In [ ]:
import pandas as pd
import numpy as np

# path_input = '../input/arabic-text-summarization-30-000/wikiHow.csv'
path_input = '../input/headlines/summary.csv'
# df['summary'] = df['summary'].replace(r'\n', '', regex = True)

# df = pd.read_csv(path_input)

df.head()

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer("../input/arabict5tokenizer/arabic_sentencepiece.model",
                               do_lower_case=True, do_basic_tokenize=True, 
                               padding=True, bos_token="<s>", 
                               eos_token="</s>",unk_token="<unk>", 
                               pad_token="<pad>")

In [ ]:
# import seaborn as sns
# from pylab import rcParams
# import matplotlib.pyplot as plt
# from matplotlib import rc

# %matplotlib inline
# %config InlineBackend.figure_format='retina'
# sns.set(style='whitegrid', palette='muted', font_scale=1.2)
# rcParams['figure.figsize'] = 16, 10

# text_token_counts = df['text'].apply(lambda x : len(tokenizer.encode(x)))
# summary_token_counts = df['summary'].apply(lambda x : len(tokenizer.encode(x)))
# fig, (ax1, ax2) = plt.subplots(1, 2)
# sns.histplot(text_token_counts, ax=ax1)
# ax1.set_title('full text token counts')
# sns.histplot(summary_token_counts, ax=ax2)
# ax2.set_title('summary text token counts')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

TEXT_MAX_LEN = 1000
SUMMARY_MAX_LEN = 100 
class SummaryDataset(Dataset):
    def __init__(
        self,
        data: pd.DataFrame = df,
        tokenizer: T5Tokenizer = tokenizer,
        text_max_token_len: int = TEXT_MAX_LEN,
        summary_max_token_len: int = SUMMARY_MAX_LEN
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        text = data_row['text']

        text_encoding = tokenizer(
            text,
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        summary_encoding = tokenizer(
            data_row['summary'],
            max_length=self.summary_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        labels = summary_encoding['input_ids']
        labels[labels == tokenizer.pad_token_id] = -100

        return dict(
            input_ids=text_encoding['input_ids'].flatten(),
            attention_mask=text_encoding['attention_mask'].flatten(),
            labels=labels.flatten(),
            decoder_attention_mask=summary_encoding['attention_mask'].flatten()
        )

dataset = SummaryDataset(df, tokenizer)
dataloader = DataLoader(dataset, shuffle=True, batch_size=4)

In [ ]:
from transformers import T5Config, T5ForConditionalGeneration, AdamW, get_scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint = torch.load('../input/retrain-t5-model-summarization/t5_Arabic_WikiHow.pth', map_location=device)

# config = T5Config(
#     vocab_size = tokenizer.vocab_size,
#     pad_token_id = tokenizer.pad_token_id,
#     eos_token_id = tokenizer.eos_token_id,
#     decoder_start_token_id = tokenizer.pad_token_id,
#     d_model = 300
# )
# model = T5ForConditionalGeneration(config)

model = checkpoint['model']
model = model.to(device)

optimizer = AdamW(model.parameters(), lr = 0.0001)
# optimizer = checkpoint['optimizer']

num_epochs = 5
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps= 30 * len(dataloader)
# )
# lr_scheduler.load_state_dict(checkpoint['lr_scheduler_state_dict'])

last_epoch = checkpoint['epoch']
print(f"{checkpoint['loss']}")

In [ ]:
from tqdm.auto import tqdm

num_training_steps = num_epochs * len(dataloader)
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        logits = outputs.logits
        
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
#         lr_scheduler.step()
        
        optimizer.zero_grad()
        progress_bar.update()
    
    torch.save({
            'epoch':  epoch + last_epoch + 1,
            'model': model,
#             'lr_scheduler_state_dict': lr_scheduler.state_dict(),
            'loss': loss,
            }, f'./t5_Arabic_WikiHow.pth')

    print(f'epoch: {epoch + last_epoch + 1} -- loss: {loss}')

In [ ]:
def summarizeText(text, model=model):
    text_encoding = tokenizer(
        text,
        max_length=TEXT_MAX_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
        
    generated_ids = model.generate(
        input_ids=text_encoding['input_ids'].to(device),
        attention_mask=text_encoding['attention_mask'].to(device),
        max_length=SUMMARY_MAX_LEN,
        num_beams=1,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )    

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)

In [ ]:
text = """
تضرر عشرات الآلاف من الركاب جراء إلغاء آلاف الرحلات الجوية، حيث تسبب الارتفاع الكبير في عدد حالات كوفيد 19 في نقص الموظفين.

وتم إلغاء أكثر من سبعة آلاف رحلة طيران منذ يوم الجمعة وعطلة نهاية الأسبوع في عطلة عيد الميلاد، وفقًا لموقع فلايت أوير الخاص بتتبع الطائرات.

ويعتقد أن شركات الطيران الصينية والأمريكية هي الأكثر تضررا، مع إعلان المزيد من التأخير والإلغاء ليوم الاثنين.

وتقول الشركات إن الإلغاء يرجع إلى إصابة الكثير من أعضاء أطقم الطائرات بكوفيد 19

كما يضطر الموظفون الذين لم تظهر إصابتهم ولكنهم على اتصال بالمصابين إلى عزل أنفسهم.
"""

ground_truth = "إلغاء رحلات عشرات الآلاف من المسافرين بسبب الوباء"

summary = summarizeText(text, model)

print(summary)